# Python API
### 1、Loading an ONNX Model

In [1]:
!ls

PythonAPI.ipynb  resources


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals


import onnx

import os

onnx_model = onnx.load(os.path.join('resources', 'logreg_iris.onnx')) # Tutorial01中的ONNX
print(type(onnx_model), id(onnx_model))
print(onnx_model)

<class 'onnx.onnx_ml_pb2.ModelProto'> 139717098346736
ir_version: 4
producer_name: "skl2onnx"
producer_version: "1.7.0"
domain: "ai.onnx"
model_version: 0
doc_string: ""
graph {
  node {
    input: "float_input"
    output: "label"
    output: "probability_tensor"
    name: "LinearClassifier"
    op_type: "LinearClassifier"
    attribute {
      name: "classlabels_ints"
      ints: 0
      ints: 1
      ints: 2
      type: INTS
    }
    attribute {
      name: "coefficients"
      floats: -0.5013933181762695
      floats: 0.8189675807952881
      floats: -2.254585027694702
      floats: -0.9972120523452759
      floats: 0.5146268606185913
      floats: -0.4061462879180908
      floats: -0.23364509642124176
      floats: -0.7229060530662537
      floats: -0.013233542442321777
      floats: -0.41282132267951965
      floats: 2.488229990005493
      floats: 1.7201180458068848
      type: FLOATS
    }
    attribute {
      name: "intercepts"
      floats: 9.711934089660645
      floats: 2

### 2、Loading an ONNX Model with External Data

[Default] If the external data is under the same directory of the model, simply use onnx.load()   
```python
import onnx

onnx_model = onnx.load('path/to/the/model.onnx')

```

If the external data is under another directory, use load_external_data_for_model() to specify the directory path and load after using onnx.load()



In [3]:
import onnx
from onnx.external_data_helper import load_external_data_for_model

onnx_model = onnx.load(os.path.join('resources', 'alexnet.onnx'), load_external_data=False)
load_external_data_for_model(onnx_model, '../unet.onnx')
# Then the onnx_model has loaded the external data from the specific directory

### 3、Saving an ONNX Model


In [4]:
# Preprocessing: load the old model
old_model_path = os.path.join('resources', 'alexnet.onnx')
onnx_model = onnx.load(old_model_path)

# Preprocessing: get the path to the saved model
new_model_path = os.path.join('./', 'alexnet.onnx')

# Save the ONNX model
onnx.save(onnx_model, new_model_path)

print('The model is saved.')

The model is saved.


### 4、Manipulating TensorProto and Numpy Array

In [5]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import numpy
import onnx
import os
from onnx import numpy_helper
from distutils.version import LooseVersion


# Preprocessing: create a Numpy array
numpy_array = numpy.array([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]], dtype=float)
if LooseVersion(numpy.version.version) < LooseVersion('1.14'):
    print('Original Numpy array:\n{}\n'.format(numpy.array2string(numpy_array)))
else:
    print('Original Numpy array:\n{}\n'.format(numpy.array2string(numpy_array, legacy='1.13')))

Original Numpy array:
[[ 1.  2.  3.]
 [ 4.  5.  6.]]



In [6]:
# Convert the Numpy array to a TensorProto
tensor = numpy_helper.from_array(numpy_array)
print('TensorProto:\n{}'.format(tensor))

TensorProto:
dims: 2
dims: 3
data_type: 11
raw_data: "\000\000\000\000\000\000\360?\000\000\000\000\000\000\000@\000\000\000\000\000\000\010@\000\000\000\000\000\000\020@\000\000\000\000\000\000\024@\000\000\000\000\000\000\030@"



In [7]:
# Convert the TensorProto to a Numpy array
new_array = numpy_helper.to_array(tensor)
if LooseVersion(numpy.version.version) < LooseVersion('1.14'):
    print('After round trip, Numpy array:\n{}\n'.format(numpy.array2string(numpy_array)))
else:
    print('After round trip, Numpy array:\n{}\n'.format(numpy.array2string(numpy_array, legacy='1.13')))

After round trip, Numpy array:
[[ 1.  2.  3.]
 [ 4.  5.  6.]]



In [8]:
# Save the TensorProto
with open(os.path.join('resources', 'tensor.pb'), 'wb') as f:
    f.write(tensor.SerializeToString())

In [9]:
# Load the TensorProto
new_tensor = onnx.TensorProto()
with open(os.path.join('resources', 'tensor.pb'), 'rb') as f:
    new_tensor.ParseFromString(f.read())
print('After saving and loading, new TensorProto:\n{}'.format(new_tensor))

After saving and loading, new TensorProto:
dims: 2
dims: 3
data_type: 11
raw_data: "\000\000\000\000\000\000\360?\000\000\000\000\000\000\000@\000\000\000\000\000\000\010@\000\000\000\000\000\000\020@\000\000\000\000\000\000\024@\000\000\000\000\000\000\030@"



### 5、Creating an ONNX Model Using Helper Functions
#### make_model.ipynb

In [10]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
from onnx import helper
from onnx import AttributeProto, TensorProto, GraphProto


# The protobuf definition can be found here:
# https://github.com/onnx/onnx/blob/master/onnx/onnx.proto


# Create one input (ValueInfoProto)
X = helper.make_tensor_value_info('X', TensorProto.FLOAT, [1, 2])

# Create second input (ValueInfoProto)
Pads = helper.make_tensor_value_info('Pads', TensorProto.INT64, [4])

# Create one output (ValueInfoProto)
Y = helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1, 4])

# Create a node (NodeProto)
node_def = helper.make_node(
    'Pad', # node name
    ['X', 'Pads'], # inputs
    ['Y'], # outputs
    mode='constant', # Attributes
)

# Create the graph (GraphProto)
graph_def = helper.make_graph(
    [node_def],
    "test-model",
    [X, Pads],
    [Y],
    [helper.make_tensor('Pads', TensorProto.INT64, [4,], [0, 0, 1, 1,])],
)

# Create the model (ModelProto)
model_def = helper.make_model(graph_def,
                              producer_name='onnx-example')

print('The ir_version in model: {}\n'.format(model_def.ir_version))
print('The producer_name in model: {}\n'.format(model_def.producer_name))
print('The graph in model:\n{}'.format(model_def.graph))
onnx.checker.check_model(model_def)
print('The model is checked!')

The ir_version in model: 7

The producer_name in model: onnx-example

The graph in model:
node {
  input: "X"
  input: "Pads"
  output: "Y"
  op_type: "Pad"
  attribute {
    name: "mode"
    s: "constant"
    type: STRING
  }
}
name: "test-model"
initializer {
  dims: 4
  data_type: 7
  int64_data: 0
  int64_data: 0
  int64_data: 1
  int64_data: 1
  name: "Pads"
}
input {
  name: "X"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 2
        }
      }
    }
  }
}
input {
  name: "Pads"
  type {
    tensor_type {
      elem_type: 7
      shape {
        dim {
          dim_value: 4
        }
      }
    }
  }
}
output {
  name: "Y"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
        dim {
          dim_value: 4
        }
      }
    }
  }
}

The model is checked!


#### Protobufs.ipynb

In [11]:
from onnx import *

In [12]:
# Int Attibute
arg = helper.make_attribute("this_is_an_int", 1701)
print("\nInt attribute:\n")
print(arg)


Int attribute:

name: "this_is_an_int"
i: 1701
type: INT



In [13]:
#NBVAL_IGNORE_OUTPUT
# Float Attribute
arg = helper.make_attribute("this_is_a_float", 3.141)
print("\nFloat attribute:\n")
print(arg)


Float attribute:

name: "this_is_a_float"
f: 3.1410000324249268
type: FLOAT



In [14]:
# String Attribute
arg = helper.make_attribute("this_is_a_string", "string_content")
print("\nString attribute:\n")
print(arg)


String attribute:

name: "this_is_a_string"
s: "string_content"
type: STRING



In [15]:
# Repeated Attribute
arg = helper.make_attribute("this_is_a_repeated_int", [1, 2, 1, 4])
print("\nRepeated int attribute:\n")
print(arg)


Repeated int attribute:

name: "this_is_a_repeated_int"
ints: 1
ints: 2
ints: 1
ints: 4
type: INTS



In [16]:
# node
node_proto = helper.make_node("Relu", ["X"], ["Y"])

print("\nNodeProto:\n")
print(node_proto)


NodeProto:

input: "X"
output: "Y"
op_type: "Relu"



In [17]:
# node with args
node_proto = helper.make_node(
    "Conv", ["X", "W", "B"], ["Y"],
    kernel=3, stride=1, pad=1)

# This is just for making the attributes to be printed in order
node_proto.attribute.sort(key=lambda attr: attr.name)
print("\nNodeProto:\n")
print(node_proto)

print("\nMore Readable NodeProto (no args yet):\n")
print(helper.printable_node(node_proto))


NodeProto:

input: "X"
input: "W"
input: "B"
output: "Y"
op_type: "Conv"
attribute {
  name: "kernel"
  i: 3
  type: INT
}
attribute {
  name: "pad"
  i: 1
  type: INT
}
attribute {
  name: "stride"
  i: 1
  type: INT
}


More Readable NodeProto (no args yet):

%Y = Conv[kernel = 3, pad = 1, stride = 1](%X, %W, %B)


In [18]:
# graph
graph_proto = helper.make_graph(
    [
        helper.make_node("FC", ["X", "W1", "B1"], ["H1"]),
        helper.make_node("Relu", ["H1"], ["R1"]),
        helper.make_node("FC", ["R1", "W2", "B2"], ["Y"]),
    ],
    "MLP",
    [
        helper.make_tensor_value_info('X' , TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W2', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B2', TensorProto.FLOAT, [1]),
    ],
    [
        helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1]),
    ]
)

print("\ngraph proto:\n")
print(graph_proto)

print("\nMore Readable GraphProto:\n")
print(helper.printable_graph(graph_proto))


graph proto:

node {
  input: "X"
  input: "W1"
  input: "B1"
  output: "H1"
  op_type: "FC"
}
node {
  input: "H1"
  output: "R1"
  op_type: "Relu"
}
node {
  input: "R1"
  input: "W2"
  input: "B2"
  output: "Y"
  op_type: "FC"
}
name: "MLP"
input {
  name: "X"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "W1"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "B1"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "W2"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
input {
  name: "B2"
  type {
    tensor_type {
      elem_type: 1
      shape {
        dim {
          dim_value: 1
        }
      }
    }
  }
}
output {
  name:

In [19]:
# An node that is also a graph
graph_proto = helper.make_graph(
    [
        helper.make_node("FC", ["X", "W1", "B1"], ["H1"]),
        helper.make_node("Relu", ["H1"], ["R1"]),
        helper.make_node("FC", ["R1", "W2", "B2"], ["Y"]),
    ],
    "MLP",
    [
        helper.make_tensor_value_info('X' , TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B1', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('W2', TensorProto.FLOAT, [1]),
        helper.make_tensor_value_info('B2', TensorProto.FLOAT, [1]),
    ],
    [
        helper.make_tensor_value_info('Y', TensorProto.FLOAT, [1]),
    ]
)

# output = ThisSpecificgraph([input, w1, b1, w2, b2])
node_proto = helper.make_node(
    "graph",
    ["Input", "W1", "B1", "W2", "B2"],
    ["Output"],
    graph=[graph_proto],
)

print("\nNodeProto that contains a graph:\n")
print(node_proto)


NodeProto that contains a graph:

input: "Input"
input: "W1"
input: "B1"
input: "W2"
input: "B2"
output: "Output"
op_type: "graph"
attribute {
  name: "graph"
  graphs {
    node {
      input: "X"
      input: "W1"
      input: "B1"
      output: "H1"
      op_type: "FC"
    }
    node {
      input: "H1"
      output: "R1"
      op_type: "Relu"
    }
    node {
      input: "R1"
      input: "W2"
      input: "B2"
      output: "Y"
      op_type: "FC"
    }
    name: "MLP"
    input {
      name: "X"
      type {
        tensor_type {
          elem_type: 1
          shape {
            dim {
              dim_value: 1
            }
          }
        }
      }
    }
    input {
      name: "W1"
      type {
        tensor_type {
          elem_type: 1
          shape {
            dim {
              dim_value: 1
            }
          }
        }
      }
    }
    input {
      name: "B1"
      type {
        tensor_type {
          elem_type: 1
          shape {
            dim

### 6、Checking an ONNX Model

In [20]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
import os


# Preprocessing: load the ONNX model
model_path = os.path.join('resources', 'alexnet.onnx')
onnx_model = onnx.load(model_path)

print('The model is:\n{}'.format(onnx_model))

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [21]:
# Check the model
onnx.checker.check_model(onnx_model)
print('The model is checked!')

The model is checked!


### 7、Checking a Large ONNX Model >2GB
Current checker supports checking models with external data, but for those models larger than 2GB, please use the model path for onnx.checker and the external data needs to be under the same directory.

In [22]:
import onnx

# onnx.checker.check_model('path/to/the/model.onnx')
# onnx.checker.check_model(loaded_onnx_model) will fail if given >2GB model
onnx.checker.check_model('resources/alexnet.onnx')

### 8、Optimizing an ONNX Model

In [25]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
import os
from onnx import optimizer

# Preprocessing: load the model contains two transposes.
model_path = os.path.join('resources', 'two_transposes.onnx')
original_model = onnx.load(model_path)

print('The model before optimization:\n\n{}'.format(onnx.helper.printable_graph(original_model.graph)))

The model before optimization:

graph two-transposes (
  %X[FLOAT, 2x3x4]
) {
  %Y = Transpose[perm = [1, 0, 2]](%X)
  %Z = Transpose[perm = [1, 0, 2]](%Y)
  return %Z
}


In [26]:
# A full list of supported optimization passes can be found using get_available_passes()
all_passes = optimizer.get_available_passes()
print("Available optimization passes:")
for p in all_passes:
    print('\t{}'.format(p))
print("end!")

# Pick one pass as example
passes = ['fuse_consecutive_transposes']

# Apply the optimization on the original serialized model
optimized_model = optimizer.optimize(original_model, passes)

print('The model after optimization:\n\n{}'.format(onnx.helper.printable_graph(optimized_model.graph)))

Available optimization passes:
	eliminate_deadend
	eliminate_identity
	eliminate_nop_dropout
	eliminate_nop_monotone_argmax
	eliminate_nop_pad
	eliminate_nop_transpose
	eliminate_unused_initializer
	extract_constant_to_initializer
	fuse_add_bias_into_conv
	fuse_bn_into_conv
	fuse_consecutive_concats
	fuse_consecutive_log_softmax
	fuse_consecutive_reduce_unsqueeze
	fuse_consecutive_squeezes
	fuse_consecutive_transposes
	fuse_matmul_add_bias_into_gemm
	fuse_pad_into_conv
	fuse_transpose_into_gemm
	lift_lexical_references
	nop
	split_init
	split_predict
end!
The model after optimization:

graph two-transposes (
  %X[FLOAT, 2x3x4]
) {
  %Z = Transpose[perm = [0, 1, 2]](%X)
  return %Z
}


### 9、Running Shape Inference on an ONNX Model

In [28]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import onnx
from onnx import helper, shape_inference
from onnx import TensorProto


# Preprocessing: create a model with two nodes, Y's shape is unknown
node1 = helper.make_node('Transpose', ['X'], ['Y'], perm=[1, 0, 2])
node2 = helper.make_node('Transpose', ['Y'], ['Z'], perm=[1, 0, 2])

graph = helper.make_graph(
    [node1, node2],
    'two-transposes',
    [helper.make_tensor_value_info('X', TensorProto.FLOAT, (2, 3, 4))],
    [helper.make_tensor_value_info('Z', TensorProto.FLOAT, (2, 3, 4))],
)

original_model = helper.make_model(graph, producer_name='onnx-examples')

# Check the model and print Y's shape information
onnx.checker.check_model(original_model)
print('Before shape inference, the shape info of Y is:\n{}'.format(original_model.graph.value_info))

Before shape inference, the shape info of Y is:
[]


In [29]:
# Apply shape inference on the model
inferred_model = shape_inference.infer_shapes(original_model)

# Check the model and print Y's shape information
onnx.checker.check_model(inferred_model)
print('After shape inference, the shape info of Y is:\n{}'.format(inferred_model.graph.value_info))

After shape inference, the shape info of Y is:
[name: "Y"
type {
  tensor_type {
    elem_type: 1
    shape {
      dim {
        dim_value: 3
      }
      dim {
        dim_value: 2
      }
      dim {
        dim_value: 4
      }
    }
  }
}
]


### 10、Converting Version of an ONNX Model within Default Domain (""/"ai.onnx")

In [6]:
import onnx
from onnx import version_converter, helper

# Preprocessing: load the model to be converted.
model_path = 'resources/alexnet2.onnx'
original_model = onnx.load(model_path)

# print('The model before conversion:\n{}'.format(original_model))

# A full list of supported adapters can be found here:
# https://github.com/onnx/onnx/blob/master/onnx/version_converter.py#L21
# Apply the version conversion on the original model
# converted_model = version_converter.convert_version(original_model, <int target_version>)
converted_model = version_converter.convert_version(original_model, 11)
print("finished")
# print('The model after conversion:\n{}'.format(converted_model))

IndexError: Input features.0.weight is undefined!

## Utility Functions
### 1、Polishing the Model

Function polish_model runs model checker, optimizer, shape inference engine on the model, and also strips the doc_string for you.



In [7]:
import torch
import torchvision

dummy_input = torch.randn(10, 3, 224, 224)
model = torchvision.models.alexnet(pretrained=True)

torch.onnx.export(model, dummy_input, "resources/alexnet2.onnx")

In [8]:
import onnx
import onnx.utils


model = onnx.load('resources/alexnet2.onnx')
polished_model = onnx.utils.polish_model(model)

IndexError: Input features.0.weight is undefined!

In [9]:
import onnx
import onnx.optimizer
model = onnx.load("resources/alexnet2.onnx")
onnx.checker.check_model(model)
inferred_model = onnx.shape_inference.infer_shapes(model)
model = onnx.optimizer.optimize(inferred_model, passes=['fuse_bn_into_conv'])

IndexError: Input features.0.weight is undefined!

## Tools

Updating Model's Inputs Outputs Dimension Sizes with Variable Length

Function update_inputs_outputs_dims updates the dimension of the inputs and outputs of the model, to the provided values in the parameter. You could provide both static and dynamic dimension size, by using dim_param. For more information on static and dynamic dimension size, checkout Tensor Shapes.

The function runs model checker after the input/output sizes are updated.

In [11]:
import onnx
from onnx.tools import update_model_dims

model = onnx.load('resources/alexnet2.onnx')
# Here both 'seq', 'batch' and -1 are dynamic using dim_param.
variable_length_model = update_model_dims.update_inputs_outputs_dims(model, {'input_name': ['seq', 'batch', 3, -1]}, {'output_name': ['seq', 'batch', 1, -1]})

KeyError: 'input.1'